In [1]:
import pandas as pd
from collections import Counter
from config_datasets import config_datasets
from cargar_dataset import cargar_dataset, graficar_distribucion_clases
from datetime import datetime
from pathlib import Path
# Crear carpeta de resultados si no existe
Path("resultados").mkdir(exist_ok=True)
Path("figuras").mkdir(exist_ok=True)

# Nombre de archivo con fecha y hora
timestamp = datetime.now().strftime("%Y-%m-%d_%H%M")
nombre_archivo = f"resultados/reporte_distribucion_{timestamp}.txt"

# Lista de líneas para guardar en el archivo
lineas_resultado = []

for nombre, cfg in config_datasets.items():
    lineas_resultado.append(f"\n🔍 Analizando dataset: {nombre.upper()}")
    print(f"\n🔍 Analizando dataset: {nombre.upper()}")
    try:
        # Cargar el dataset (multiclase)
        X, y, _ = cargar_dataset(
            path=cfg["path"],
            clase_minoria=cfg.get("clase_minoria"),
            col_features=cfg.get("col_features"),
            col_target=cfg.get("col_target"),
            sep=cfg.get("sep", ","),
            header=cfg.get("header"),
            binarizar=cfg.get("binarizar", False),
            tipo=cfg.get("tipo", "tabular"),
            dataset_name=cfg.get("dataset_name")
        )


        # Contar clases originales
        conteo = pd.Series(y).value_counts()
        clase_min_real = conteo.idxmin()
        total = conteo.sum()
        proporcion = (conteo / total * 100).round(2)

        # Mostrar
        print("🎯 Valores únicos del target:", list(conteo.index))
        print("📊 Distribución de clases:")
        # Guardar resultados
        lineas_resultado.append(f"🎯 Valores únicos del target: {list(conteo.index)}")
        lineas_resultado.append("📊 Distribución de clases:")

        for clase, count in conteo.items():
            print(f"   - {clase}: {count} ({proporcion[clase]}%)")
            lineas_resultado.append(f"   - {clase}: {count} ({proporcion[clase]}%)")

        print(f"✅ Clase minoritaria real: {clase_min_real}")
        print(f"⚠️ Clase configurada como minoritaria: {cfg['clase_minoria']}")
        lineas_resultado.append(f"✅ Clase minoritaria real: {clase_min_real}")
        lineas_resultado.append(f"⚠️ Clase configurada como minoritaria: {cfg.get('clase_minoria')}")

        if "clase_minoria" in cfg and cfg["clase_minoria"] is not None:
            if clase_min_real != cfg["clase_minoria"]:
                print("🚨 POSIBLE ERROR DE CONFIGURACIÓN ❗")
                lineas_resultado.append("🚨 POSIBLE ERROR DE CONFIGURACIÓN ❗")
        else:
            print("ℹ️ No se definió clase minoritaria (modo multiclase o imagen).")
            lineas_resultado.append("ℹ️ No se definió clase minoritaria (modo multiclase o imagen).")

        # 🔽 Agregar gráfico descriptivo por dataset
        nombre_figura = f"figuras/{nombre.lower()}_distribucion_{timestamp}.png"
        graficar_distribucion_clases(y, nombre_dataset=nombre, guardar_en=nombre_figura)

    except Exception as e:
        print(f"❌ Error al analizar {nombre}: {e}")
        lineas_resultado.append(f"❌ Error al analizar {nombre}: {e}")

# Guardar a archivo
with open(nombre_archivo, "w", encoding="utf-8") as f:
    f.write("\n".join(lineas_resultado))

print(f"\n📁 Análisis guardado en: {nombre_archivo}")



🔍 Analizando dataset: SHUTTLE
❌ Error al analizar shuttle: "None of [Index(['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'], dtype='object')] are in the [columns]"

🔍 Analizando dataset: WDBC
🎯 Valores únicos del target: ['B', 'M']
📊 Distribución de clases:
   - B: 357 (62.74%)
   - M: 212 (37.26%)
✅ Clase minoritaria real: M
⚠️ Clase configurada como minoritaria: M

🔍 Analizando dataset: GLASS
🎯 Valores únicos del target: [2, 1, 7, 3, 5, 6]
📊 Distribución de clases:
   - 2: 76 (35.51%)
   - 1: 70 (32.71%)
   - 7: 29 (13.55%)
   - 3: 17 (7.94%)
   - 5: 13 (6.07%)
   - 6: 9 (4.21%)
✅ Clase minoritaria real: 6
⚠️ Clase configurada como minoritaria: 6

🔍 Analizando dataset: HEART
🎯 Valores únicos del target: [0, 1, 2, 3, 4]
📊 Distribución de clases:
   - 0: 164 (54.13%)
   - 1: 55 (18.15%)
   - 2: 36 (11.88%)
   - 3: 35 (11.55%)
   - 4: 13 (4.29%)
✅ Clase minoritaria real: 4
⚠️ Clase configurada como minoritaria: 4

🔍 Analizando dataset: IRIS
🎯 Valores únicos del target: ['Iris-set

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import shapiro, skew, kurtosis

# --- Utilidades numéricas seguras ---
def es_numerica(serie):
    return np.issubdtype(serie.dtype, np.number)

def obtener_matriz_correlacion_segura(df_numerico):
    # Evita NaNs en correlación
    df_sin_nan = df_numerico.fillna(df_numerico.median(numeric_only=True))
    return df_sin_nan.corr().values, list(df_numerico.columns)

# --- Métricas de calidad del dataset ---
def calcular_metricas_basicas_dataframe(X_df, y_series):
    metricas = {}

    # Filtrado numérico
    columnas_numericas = [c for c in X_df.columns if es_numerica(X_df[c])]
    X_num = X_df[columnas_numericas].copy()

    # Tamaños básicos
    metricas["cantidad_muestras"] = int(X_df.shape[0])
    metricas["cantidad_atributos"] = int(X_df.shape[1])

    # Faltantes y duplicados
    total_celdas = int(X_df.shape[0] * X_df.shape[1])
    cantidad_faltantes = int(X_df.isnull().sum().sum())
    metricas["porcentaje_faltantes"] = float((cantidad_faltantes / total_celdas) * 100.0)

    cantidad_duplicados = int(X_df.duplicated().sum())
    metricas["porcentaje_duplicados"] = float((cantidad_duplicados / X_df.shape[0]) * 100.0)

    # Distribución de clases
    valores_unicos, conteos = np.unique(y_series, return_counts=True)
    cantidad_clases = int(len(valores_unicos))
    indice_min = int(np.argmin(conteos))
    indice_max = int(np.argmax(conteos))
    clase_min = valores_unicos[indice_min]
    clase_max = valores_unicos[indice_max]
    n_min = int(conteos[indice_min])
    n_max = int(conteos[indice_max])

    metricas["cantidad_clases"] = cantidad_clases
    metricas["clase_minima_real"] = str(clase_min)
    metricas["clase_mayoritaria_real"] = str(clase_max)
    metricas["n_min"] = n_min
    metricas["n_max"] = n_max
    metricas["ratio_desequilibrio_max"] = float(n_max / max(1, n_min))

    # Entropía de clases y tamaño efectivo
    proporciones = conteos / conteos.sum()
    entropia = float(-(proporciones * np.log(proporciones + 1e-12)).sum())
    metricas["entropia_clases"] = entropia
    metricas["tamano_efectivo_clases"] = float(np.exp(entropia))  # Effective Number of Classes

    # Rango de escalas por columna (para sugerir scaler)
    rangos = []
    for nombre_col in columnas_numericas:
        col = X_num[nombre_col].dropna()
        if col.shape[0] > 0:
            valor_min = float(np.min(col))
            valor_max = float(np.max(col))
            rango = float(valor_max - valor_min)
            rangos.append(rango)
    if len(rangos) > 0:
        metricas["rango_mediano_variables"] = float(np.median(rangos))
        metricas["rango_maximo_variables"] = float(np.max(rangos))
    else:
        metricas["rango_mediano_variables"] = 0.0
        metricas["rango_maximo_variables"] = 0.0

    # Asimetría y curtosis agregadas
    skew_acumulado = []
    kurt_acumulado = []
    for nombre_col in columnas_numericas:
        col = X_num[nombre_col].dropna().astype(float)
        if col.shape[0] > 3:
            skew_acumulado.append(float(skew(col)))
            kurt_acumulado.append(float(kurtosis(col, fisher=True)))
    if len(skew_acumulado) > 0:
        metricas["asimetria_mediana"] = float(np.median(skew_acumulado))
        metricas["curtosis_mediana"] = float(np.median(kurt_acumulado))
    else:
        metricas["asimetria_mediana"] = 0.0
        metricas["curtosis_mediana"] = 0.0

    # Normalidad (Shapiro-Wilk sobre una muestra si N>5000)
    # Devuelve porcentaje de variables con p>0.05 (no se rechaza normalidad)
    porcentaje_normalidad = 0.0
    variables_evaluadas = 0
    for nombre_col in columnas_numericas:
        col = X_num[nombre_col].dropna().astype(float)
        tam_col = col.shape[0]
        if tam_col > 3:
            if tam_col > 5000:
                # muestreo simple para evitar límites de Shapiro
                col = col.sample(5000, random_state=123).astype(float)
            try:
                estadistico, pvalor = shapiro(col.values)
                variables_evaluadas += 1
                if pvalor > 0.05:
                    porcentaje_normalidad += 1.0
            except Exception:
                # si Shapiro falla en alguna variable, la omitimos
                pass
    if variables_evaluadas > 0:
        metricas["porcentaje_variables_con_normalidad_no_rechazada"] = float((porcentaje_normalidad / variables_evaluadas) * 100.0)
    else:
        metricas["porcentaje_variables_con_normalidad_no_rechazada"] = 0.0

    # Correlación: máximo |r| y % de pares con |r| > 0.9
    if X_num.shape[1] >= 2:
        matriz_corr, nombres = obtener_matriz_correlacion_segura(X_num)
        valores_superior = []
        cantidad_altamente_correl = 0
        total_pares = 0

        for i in range(len(nombres)):
            for j in range(i + 1, len(nombres)):
                r = float(matriz_corr[i, j])
                valores_superior.append(abs(r))
                total_pares += 1
                if abs(r) > 0.9:
                    cantidad_altamente_correl += 1

        if len(valores_superior) > 0:
            metricas["correlacion_absoluta_maxima"] = float(np.max(valores_superior))
            metricas["porcentaje_pares_correlacion_mayor_0_9"] = float((cantidad_altamente_correl / total_pares) * 100.0)
        else:
            metricas["correlacion_absoluta_maxima"] = 0.0
            metricas["porcentaje_pares_correlacion_mayor_0_9"] = 0.0
    else:
        metricas["correlacion_absoluta_maxima"] = 0.0
        metricas["porcentaje_pares_correlacion_mayor_0_9"] = 0.0

    # Sugerencia de escalado (reglas simples y explícitas)
    sugerencia_escalado = "MinMaxScaler"
    if metricas["porcentaje_variables_con_normalidad_no_rechazada"] > 60.0 and metricas["correlacion_absoluta_maxima"] < 0.95:
        sugerencia_escalado = "StandardScaler"
    if metricas["asimetria_mediana"] > 1.0 or metricas["curtosis_mediana"] > 1.0:
        sugerencia_escalado = "RobustScaler"
    metricas["sugerencia_escalado"] = sugerencia_escalado

    return metricas

# --- Detección de outliers por IQR ---
def calcular_porcentaje_outliers_por_variable(X_df):
    resultado = {}
    for nombre_columna in X_df.columns:
        if es_numerica(X_df[nombre_columna]):
            serie = X_df[nombre_columna].dropna().astype(float)
            if serie.shape[0] > 0:
                q1 = float(np.percentile(serie, 25))
                q3 = float(np.percentile(serie, 75))
                iqr = float(q3 - q1)
                limite_inferior = q1 - 1.5 * iqr
                limite_superior = q3 + 1.5 * iqr

                cantidad = int(serie.shape[0])
                cantidad_out = 0
                indice = 0
                valores = serie.values
                while indice < cantidad:
                    valor_actual = float(valores[indice])
                    if valor_actual < limite_inferior or valor_actual > limite_superior:
                        cantidad_out += 1
                    indice += 1
                porcentaje = float((cantidad_out / cantidad) * 100.0)
                resultado[nombre_columna] = porcentaje
    return resultado

# --- Gráficos simples y guardado ---
def graficar_histograma_variables(X_df, nombre_dataset, ruta_base, max_columnas=12):
    # Toma las primeras N numéricas para no explotar la figura
    columnas = [c for c in X_df.columns if es_numerica(X_df[c])]
    columnas = columnas[:max_columnas]
    for nombre_col in columnas:
        plt.figure()
        valores = X_df[nombre_col].dropna().values.astype(float)
        plt.hist(valores, bins=30)
        plt.title(f"{nombre_dataset} · Histograma: {nombre_col}")
        plt.xlabel(nombre_col)
        plt.ylabel("Frecuencia")
        ruta_salida = f"{ruta_base}/{nombre_dataset}_hist_{nombre_col}.png"
        plt.tight_layout()
        plt.savefig(ruta_salida, dpi=140)
        plt.close()

def graficar_boxplot_variables(X_df, nombre_dataset, ruta_base, max_columnas=12):
    columnas = []
    for c in X_df.columns:
        if es_numerica(X_df[c]):
            columnas.append(c)
    # Limitar cantidad para no generar demasiadas figuras
    if len(columnas) > max_columnas:
        columnas = columnas[:max_columnas]

    for nombre_col in columnas:
        # Preparar datos
        serie = X_df[nombre_col].dropna()
        valores = serie.values.astype(float)

        # Crear figura
        plt.figure()
        # 1) NO pasamos labels/tick_labels (evita deprecations/errores entre versiones)
        try:
            plt.boxplot(valores, vert=True, showfliers=True)
        except Exception as e:
            # Fallback: dibujar un punto para no romper el loop
            plt.plot([1], [valores[0] if valores.size>0 else 0], marker="o")

        # 2) Setear el tick X manualmente (compatible 100%)
        plt.xticks([1], [nombre_col])

        # 3) Título y guardado
        plt.title(f"{nombre_dataset} · Boxplot: {nombre_col}")
        ruta_salida = f"{ruta_base}/{nombre_dataset}_box_{nombre_col}.png"
        plt.tight_layout()
        plt.savefig(ruta_salida, dpi=140)
        plt.close()


def graficar_mapa_correlaciones(X_df, nombre_dataset, ruta_base):
    columnas = [c for c in X_df.columns if es_numerica(X_df[c])]
    if len(columnas) < 2:
        return
    X_num = X_df[columnas].fillna(X_df[columnas].median(numeric_only=True))
    matriz_corr = X_num.corr().values

    plt.figure()
    plt.imshow(matriz_corr, aspect='auto', interpolation='nearest')
    plt.colorbar()
    plt.xticks(np.arange(len(columnas)), columnas, rotation=90)
    plt.yticks(np.arange(len(columnas)), columnas)
    plt.title(f"{nombre_dataset} · Matriz de correlaciones (Pearson)")
    plt.tight_layout()
    ruta_salida = f"{ruta_base}/{nombre_dataset}_correlaciones.png"
    plt.savefig(ruta_salida, dpi=140)
    plt.close()


In [3]:
import csv

ruta_metricas_csv = f"resultados/metricas_eda_{timestamp}.csv"
encabezados_csv = [
    "dataset","cantidad_muestras","cantidad_atributos","cantidad_clases",
    "clase_minima_real","clase_mayoritaria_real","n_min","n_max","ratio_desequilibrio_max",
    "entropia_clases","tamano_efectivo_clases","porcentaje_faltantes","porcentaje_duplicados",
    "asimetria_mediana","curtosis_mediana","porcentaje_variables_con_normalidad_no_rechazada",
    "correlacion_absoluta_maxima","porcentaje_pares_correlacion_mayor_0_9",
    "rango_mediano_variables","rango_maximo_variables","sugerencia_escalado"
]
with open(ruta_metricas_csv, "w", newline="", encoding="utf-8") as fcsv:
    escritor = csv.writer(fcsv)
    escritor.writerow(encabezados_csv)

for nombre, cfg in config_datasets.items():
    lineas_resultado.append(f"\n🔎 Análisis extendido: {nombre.upper()}")
    print(f"\n🔎 Análisis extendido: {nombre.upper()}")
    try:
        # === Cargar dataset (tu misma función) ===
        X, y, _ = cargar_dataset(
            path=cfg["path"],
            clase_minoria=cfg.get("clase_minoria"),
            col_features=cfg.get("col_features"),
            col_target=cfg.get("col_target"),
            sep=cfg.get("sep", ","),
            header=cfg.get("header"),
            binarizar=cfg.get("binarizar", False),
            tipo=cfg.get("tipo", "tabular"),
            dataset_name=cfg.get("dataset_name")
        )

        # === Reporte de clases (ya lo haces) ===
        conteo = pd.Series(y).value_counts()
        clase_min_real = conteo.idxmin()
        total = int(conteo.sum())
        proporcion = (conteo / total * 100).round(2)

        lineas_resultado.append(f"🎯 Valores únicos del target: {list(conteo.index)}")
        lineas_resultado.append("📊 Distribución de clases:")
        for clase, count in conteo.items():
            lineas_resultado.append(f"   - {clase}: {int(count)} ({float(proporcion[clase])}%)")

        lineas_resultado.append(f"✅ Clase minoritaria real: {clase_min_real}")
        lineas_resultado.append(f"⚠️ Clase configurada como minoritaria: {cfg.get('clase_minoria')}")

        if "clase_minoria" in cfg and cfg["clase_minoria"] is not None:
            if clase_min_real != cfg["clase_minoria"]:
                aviso = "🚨 POSIBLE ERROR DE CONFIGURACIÓN ❗"
                print(aviso)
                lineas_resultado.append(aviso)
        else:
            info = "ℹ️ No se definió clase minoritaria (modo multiclase o imagen)."
            print(info)
            lineas_resultado.append(info)

        # === Métricas de calidad globales ===
        X_df = pd.DataFrame(X, columns=[f"col_{i}" for i in range(X.shape[1])]) if not isinstance(X, pd.DataFrame) else X.copy()
        y_series = pd.Series(y)

        metricas = calcular_metricas_basicas_dataframe(X_df, y_series)
        lineas_resultado.append(f"🧪 Métricas clave: {metricas}")

        # Guardar métricas al CSV
        with open(ruta_metricas_csv, "a", newline="", encoding="utf-8") as fcsv:
            escritor = csv.writer(fcsv)
            fila = [
                nombre,
                metricas["cantidad_muestras"],
                metricas["cantidad_atributos"],
                metricas["cantidad_clases"],
                metricas["clase_minima_real"],
                metricas["clase_mayoritaria_real"],
                metricas["n_min"],
                metricas["n_max"],
                round(metricas["ratio_desequilibrio_max"], 4),
                round(metricas["entropia_clases"], 4),
                round(metricas["tamano_efectivo_clases"], 4),
                round(metricas["porcentaje_faltantes"], 2),
                round(metricas["porcentaje_duplicados"], 2),
                round(metricas["asimetria_mediana"], 3),
                round(metricas["curtosis_mediana"], 3),
                round(metricas["porcentaje_variables_con_normalidad_no_rechazada"], 2),
                round(metricas["correlacion_absoluta_maxima"], 3),
                round(metricas["porcentaje_pares_correlacion_mayor_0_9"], 2),
                round(metricas["rango_mediano_variables"], 3),
                round(metricas["rango_maximo_variables"], 3),
                metricas["sugerencia_escalado"]
            ]
            escritor.writerow(fila)

        # === Outliers por IQR (resumen) ===
        porcentajes_outliers = calcular_porcentaje_outliers_por_variable(X_df)
        # Reporta top 8 variables con mayor % de outliers
        pares_ordenados = sorted(porcentajes_outliers.items(), key=lambda p: p[1], reverse=True)
        top_out = pares_ordenados[:8]
        lineas_resultado.append("📌 Top variables con mayor porcentaje de outliers (IQR):")
        for nombre_col, pct in top_out:
            lineas_resultado.append(f"   - {nombre_col}: {round(pct,2)}%")

        # === Figuras: hist, box, correlación ===
        carpeta_figuras_dataset = f"figuras/{nombre.lower()}_{timestamp}"
        Path(carpeta_figuras_dataset).mkdir(exist_ok=True)

        graficar_distribucion_clases(y, nombre_dataset=nombre, guardar_en=f"{carpeta_figuras_dataset}/{nombre.lower()}_clases.png")
        graficar_histograma_variables(X_df, nombre, carpeta_figuras_dataset, max_columnas=12)
        graficar_boxplot_variables(X_df, nombre, carpeta_figuras_dataset, max_columnas=12)
        graficar_mapa_correlaciones(X_df, nombre, carpeta_figuras_dataset)

    except Exception as e:
        mensaje_error = f"❌ Error al analizar {nombre}: {e}"
        print(mensaje_error)
        lineas_resultado.append(mensaje_error)



🔎 Análisis extendido: SHUTTLE
❌ Error al analizar shuttle: "None of [Index(['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'], dtype='object')] are in the [columns]"

🔎 Análisis extendido: WDBC

🔎 Análisis extendido: GLASS

🔎 Análisis extendido: HEART

🔎 Análisis extendido: IRIS
ℹ️ No se definió clase minoritaria (modo multiclase o imagen).

🔎 Análisis extendido: ECOLI


In [4]:
# === REPORTE VISUAL UNIFICADO (HTML) ===
import os
import csv
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
from sklearn.decomposition import PCA
from scipy.stats import shapiro, skew, kurtosis

# -------------------------------------------------------------------
# Configuración general de salida
# -------------------------------------------------------------------
timestamp = datetime.now().strftime("%Y-%m-%d_%H%M")
carpeta_resultados = Path(f"resultados_reporte_unificado_{timestamp}")
carpeta_figuras = carpeta_resultados / "figuras"
carpeta_resultados.mkdir(exist_ok=True, parents=True)
carpeta_figuras.mkdir(exist_ok=True, parents=True)

# -------------------------------------------------------------------
# Utilidades base
# -------------------------------------------------------------------
def es_numerica(serie):
    return np.issubdtype(serie.dtype, np.number)

def obtener_matriz_correlacion_segura(df_numerico):
    df_sin_nan = df_numerico.fillna(df_numerico.median(numeric_only=True))
    return df_sin_nan.corr().values, list(df_numerico.columns)

def calcular_metricas_basicas_dataframe(X_df, y_series):
    metricas = {}
    columnas_numericas = []
    for nombre_col in X_df.columns:
        if es_numerica(X_df[nombre_col]):
            columnas_numericas.append(nombre_col)
    X_num = X_df[columnas_numericas].copy()

    metricas["cantidad_muestras"] = int(X_df.shape[0])
    metricas["cantidad_atributos"] = int(X_df.shape[1])

    total_celdas = int(X_df.shape[0] * X_df.shape[1])
    cantidad_faltantes = int(X_df.isnull().sum().sum())
    metricas["porcentaje_faltantes"] = float((cantidad_faltantes / total_celdas) * 100.0) if total_celdas > 0 else 0.0

    cantidad_duplicados = int(X_df.duplicated().sum())
    metricas["porcentaje_duplicados"] = float((cantidad_duplicados / X_df.shape[0]) * 100.0) if X_df.shape[0] > 0 else 0.0

    valores_unicos, conteos = np.unique(y_series, return_counts=True)
    cantidad_clases = int(len(valores_unicos))
    indice_min = int(np.argmin(conteos))
    indice_max = int(np.argmax(conteos))
    clase_min = valores_unicos[indice_min] if cantidad_clases > 0 else ""
    clase_max = valores_unicos[indice_max] if cantidad_clases > 0 else ""
    n_min = int(conteos[indice_min]) if cantidad_clases > 0 else 0
    n_max = int(conteos[indice_max]) if cantidad_clases > 0 else 0

    metricas["cantidad_clases"] = cantidad_clases
    metricas["clase_minima_real"] = str(clase_min)
    metricas["clase_mayoritaria_real"] = str(clase_max)
    metricas["n_min"] = n_min
    metricas["n_max"] = n_max
    metricas["ratio_desequilibrio_max"] = float(n_max / max(1, n_min)) if n_min > 0 else float("inf")

    proporciones = conteos / conteos.sum() if conteos.sum() > 0 else np.array([1.0])
    entropia = float(-(proporciones * np.log(proporciones + 1e-12)).sum())
    metricas["entropia_clases"] = entropia
    metricas["tamano_efectivo_clases"] = float(np.exp(entropia))

    rangos = []
    for nombre_col in columnas_numericas:
        col = X_num[nombre_col].dropna()
        if col.shape[0] > 0:
            valor_min = float(np.min(col))
            valor_max = float(np.max(col))
            rango = float(valor_max - valor_min)
            rangos.append(rango)
    if len(rangos) > 0:
        metricas["rango_mediano_variables"] = float(np.median(rangos))
        metricas["rango_maximo_variables"] = float(np.max(rangos))
    else:
        metricas["rango_mediano_variables"] = 0.0
        metricas["rango_maximo_variables"] = 0.0

    skew_acumulado = []
    kurt_acumulado = []
    for nombre_col in columnas_numericas:
        col = X_num[nombre_col].dropna().astype(float)
        if col.shape[0] > 3:
            skew_acumulado.append(float(skew(col)))
            kurt_acumulado.append(float(kurtosis(col, fisher=True)))
    metricas["asimetria_mediana"] = float(np.median(skew_acumulado)) if len(skew_acumulado) > 0 else 0.0
    metricas["curtosis_mediana"] = float(np.median(kurt_acumulado)) if len(kurt_acumulado) > 0 else 0.0

    porcentaje_normalidad = 0.0
    variables_evaluadas = 0
    for nombre_col in columnas_numericas:
        col = X_num[nombre_col].dropna().astype(float)
        tam_col = col.shape[0]
        if tam_col > 3:
            if tam_col > 5000:
                col = col.sample(5000, random_state=123).astype(float)
            try:
                estadistico, pvalor = shapiro(col.values)
                variables_evaluadas += 1
                if pvalor > 0.05:
                    porcentaje_normalidad += 1.0
            except Exception:
                pass
    if variables_evaluadas > 0:
        metricas["porcentaje_variables_con_normalidad_no_rechazada"] = float((porcentaje_normalidad / variables_evaluadas) * 100.0)
    else:
        metricas["porcentaje_variables_con_normalidad_no_rechazada"] = 0.0

    if X_num.shape[1] >= 2:
        matriz_corr, nombres = obtener_matriz_correlacion_segura(X_num)
        valores_superior = []
        cantidad_altamente_correl = 0
        total_pares = 0
        for i in range(len(nombres)):
            for j in range(i + 1, len(nombres)):
                r = float(matriz_corr[i, j])
                valores_superior.append(abs(r))
                total_pares += 1
                if abs(r) > 0.9:
                    cantidad_altamente_correl += 1
        if len(valores_superior) > 0:
            metricas["correlacion_absoluta_maxima"] = float(np.max(valores_superior))
            metricas["porcentaje_pares_correlacion_mayor_0_9"] = float((cantidad_altamente_correl / total_pares) * 100.0)
        else:
            metricas["correlacion_absoluta_maxima"] = 0.0
            metricas["porcentaje_pares_correlacion_mayor_0_9"] = 0.0
    else:
        metricas["correlacion_absoluta_maxima"] = 0.0
        metricas["porcentaje_pares_correlacion_mayor_0_9"] = 0.0

    sugerencia_escalado = "MinMaxScaler"
    if metricas["porcentaje_variables_con_normalidad_no_rechazada"] > 60.0 and metricas["correlacion_absoluta_maxima"] < 0.95:
        sugerencia_escalado = "StandardScaler"
    if metricas["asimetria_mediana"] > 1.0 or metricas["curtosis_mediana"] > 1.0:
        sugerencia_escalado = "RobustScaler"
    metricas["sugerencia_escalado"] = sugerencia_escalado

    return metricas

def calcular_porcentaje_outliers_por_variable(X_df):
    resultado = {}
    for nombre_columna in X_df.columns:
        if es_numerica(X_df[nombre_columna]):
            serie = X_df[nombre_columna].dropna().astype(float)
            if serie.shape[0] > 0:
                q1 = float(np.percentile(serie, 25))
                q3 = float(np.percentile(serie, 75))
                iqr = float(q3 - q1)
                limite_inferior = q1 - 1.5 * iqr
                limite_superior = q3 + 1.5 * iqr
                cantidad = int(serie.shape[0])
                cantidad_out = 0
                indice = 0
                valores = serie.values
                while indice < cantidad:
                    valor_actual = float(valores[indice])
                    if valor_actual < limite_inferior or valor_actual > limite_superior:
                        cantidad_out += 1
                    indice += 1
                porcentaje = float((cantidad_out / cantidad) * 100.0)
                resultado[nombre_columna] = porcentaje
    return resultado

# -------------------------------------------------------------------
# Gráficos (solo Matplotlib) — compatibles con cualquier versión
# -------------------------------------------------------------------
def graficar_histograma_variables(X_df, nombre_dataset, ruta_base, max_columnas=12):
    columnas_numericas = []
    for c in X_df.columns:
        if es_numerica(X_df[c]):
            columnas_numericas.append(c)
    if len(columnas_numericas) > max_columnas:
        columnas_numericas = columnas_numericas[:max_columnas]
    for nombre_col in columnas_numericas:
        plt.figure()
        valores = X_df[nombre_col].dropna().values.astype(float)
        plt.hist(valores, bins=30)
        plt.title(f"{nombre_dataset} · Histograma: {nombre_col}")
        plt.xlabel(nombre_col)
        plt.ylabel("Frecuencia")
        ruta_salida = os.path.join(ruta_base, f"{nombre_dataset}_hist_{nombre_col}.png")
        plt.tight_layout()
        plt.savefig(ruta_salida, dpi=140)
        plt.close()

def graficar_boxplot_variables(X_df, nombre_dataset, ruta_base, max_columnas=12):
    columnas = []
    for c in X_df.columns:
        if es_numerica(X_df[c]):
            columnas.append(c)
    if len(columnas) > max_columnas:
        columnas = columnas[:max_columnas]
    for nombre_col in columnas:
        plt.figure()
        valores = X_df[nombre_col].dropna().values.astype(float)
        try:
            plt.boxplot(valores, vert=True, showfliers=True)  # sin labels/tick_labels
        except Exception:
            if valores.size > 0:
                plt.plot([1], [valores[0]], marker="o")
        plt.xticks([1], [nombre_col])  # seteamos el tick manualmente
        plt.title(f"{nombre_dataset} · Boxplot: {nombre_col}")
        ruta_salida = os.path.join(ruta_base, f"{nombre_dataset}_box_{nombre_col}.png")
        plt.tight_layout()
        plt.savefig(ruta_salida, dpi=140)
        plt.close()

def graficar_mapa_correlaciones(X_df, nombre_dataset, ruta_base):
    columnas = [c for c in X_df.columns if es_numerica(X_df[c])]
    if len(columnas) < 2:
        return
    X_num = X_df[columnas].fillna(X_df[columnas].median(numeric_only=True))
    matriz_corr = X_num.corr().values
    plt.figure()
    plt.imshow(matriz_corr, aspect='auto', interpolation='nearest')
    plt.colorbar()
    # ticks compatibles con cualquier versión
    posiciones = np.arange(len(columnas))
    plt.xticks(posiciones, columnas, rotation=90)
    plt.yticks(posiciones, columnas)
    plt.title(f"{nombre_dataset} · Matriz de correlaciones (Pearson)")
    plt.tight_layout()
    ruta_salida = os.path.join(ruta_base, f"{nombre_dataset}_correlaciones.png")
    plt.savefig(ruta_salida, dpi=140)
    plt.close()

# -------------------------------------------------------------------
# PCA (varianza explicada)
# -------------------------------------------------------------------
def calcular_varianza_explicada_pca(X_df, k=5):
    columnas_num = []
    for c in X_df.columns:
        if es_numerica(X_df[c]):
            columnas_num.append(c)
    if len(columnas_num) < 2:
        return []
    X_num = X_df[columnas_num].fillna(X_df[columnas_num].median(numeric_only=True)).astype(float)
    componentes = int(min(k, X_num.shape[1]))
    pca = PCA(n_components=componentes, random_state=123)
    pca.fit(X_num)
    lista_var = []
    for v in pca.explained_variance_ratio_:
        lista_var.append(float(v))
    return lista_var

# -------------------------------------------------------------------
# Generación del reporte HTML por dataset
# -------------------------------------------------------------------
def generar_html_dataset(nombre_dataset, resumen_clases, metricas, varianza_pca, rutas_figuras, ruta_html_salida):
    # Construcción manual del HTML (sin librerías externas)
    html = []
    html.append("<html><head><meta charset='utf-8'><title>Reporte " + str(nombre_dataset) + "</title>")
    html.append("<style>body{font-family:Arial,Helvetica,sans-serif;max-width:1100px;margin:24px auto;padding:0 12px;}h1{margin-bottom:6px;}h2{margin-top:28px;}table{border-collapse:collapse;width:100%;}table,th,td{border:1px solid #ddd;}th,td{padding:8px;text-align:left;}code{background:#f5f5f5;padding:2px 4px;border-radius:4px;}</style>")
    html.append("</head><body>")
    html.append("<h1>Reporte de análisis — " + str(nombre_dataset) + "</h1>")

    # Resumen de clases
    html.append("<h2>Distribución de clases</h2>")
    html.append("<table><thead><tr><th>Clase</th><th>Conteo</th><th>Proporción (%)</th></tr></thead><tbody>")
    for fila in resumen_clases:  # lista de tuplas (clase, n, pct)
        html.append("<tr><td>" + str(fila[0]) + "</td><td>" + str(fila[1]) + "</td><td>" + str(round(fila[2],2)) + "</td></tr>")
    html.append("</tbody></table>")

    # Métricas clave
    html.append("<h2>Métricas clave del dataset</h2>")
    html.append("<table><tbody>")
    for k in [
        "cantidad_muestras","cantidad_atributos","cantidad_clases",
        "clase_minima_real","clase_mayoritaria_real","n_min","n_max","ratio_desequilibrio_max",
        "entropia_clases","tamano_efectivo_clases","porcentaje_faltantes","porcentaje_duplicados",
        "asimetria_mediana","curtosis_mediana","porcentaje_variables_con_normalidad_no_rechazada",
        "correlacion_absoluta_maxima","porcentaje_pares_correlacion_mayor_0_9",
        "rango_mediano_variables","rango_maximo_variables","sugerencia_escalado"
    ]:
        html.append("<tr><th>"+str(k)+"</th><td>"+str(metricas.get(k,""))+"</td></tr>")
    html.append("</tbody></table>")

    # PCA
    html.append("<h2>PCA — Varianza explicada</h2>")
    if len(varianza_pca) > 0:
        suma_2 = sum(varianza_pca[:2])
        suma_3 = sum(varianza_pca[:3])
        html.append("<p>PC1: <b>"+str(round(varianza_pca[0],3))+"</b> · PC1+PC2: <b>"+str(round(suma_2,3))+"</b> · PC1+PC2+PC3: <b>"+str(round(suma_3,3))+"</b></p>")
        html.append("<table><thead><tr><th>Componente</th><th>Varianza explicada</th></tr></thead><tbody>")
        indice = 0
        while indice < len(varianza_pca):
            html.append("<tr><td>PC"+str(indice+1)+"</td><td>"+str(round(varianza_pca[indice],6))+"</td></tr>")
            indice += 1
        html.append("</tbody></table>")
    else:
        html.append("<p>Insuficientes columnas numéricas para PCA.</p>")

    # Imágenes
    html.append("<h2>Figuras</h2>")
    for titulo, ruta in rutas_figuras:
        rel = os.path.relpath(ruta, start=str(carpeta_resultados))
        html.append("<h3>"+titulo+"</h3>")
        html.append("<img src='"+rel+"' style='max-width:100%;height:auto;border:1px solid #ddd;padding:4px;' />")

    html.append("</body></html>")

    with open(ruta_html_salida, "w", encoding="utf-8") as f:
        f.write("\n".join(html))

# -------------------------------------------------------------------
# Orquestador principal
# -------------------------------------------------------------------
def generar_reporte_unificado(config_datasets, cargar_dataset, graficar_distribucion_clases):
    ruta_index = carpeta_resultados / "index.html"
    filas_index = []
    filas_index.append("<html><head><meta charset='utf-8'><title>Reporte unificado</title>")
    filas_index.append("<style>body{font-family:Arial,Helvetica,sans-serif;max-width:900px;margin:24px auto;padding:0 12px;}ul{line-height:1.8}</style>")
    filas_index.append("</head><body>")
    filas_index.append("<h1>Reporte unificado — Análisis exploratorio</h1>")
    filas_index.append("<ul>")

    # CSV de métricas consolidado
    ruta_metricas_csv = carpeta_resultados / "metricas_eda_consolidado.csv"
    with open(ruta_metricas_csv, "w", newline="", encoding="utf-8") as fcsv:
        escritor = csv.writer(fcsv)
        escritor.writerow([
            "dataset","cantidad_muestras","cantidad_atributos","cantidad_clases",
            "clase_minima_real","clase_mayoritaria_real","n_min","n_max","ratio_desequilibrio_max",
            "entropia_clases","tamano_efectivo_clases","porcentaje_faltantes","porcentaje_duplicados",
            "asimetria_mediana","curtosis_mediana","porcentaje_variables_con_normalidad_no_rechazada",
            "correlacion_absoluta_maxima","porcentaje_pares_correlacion_mayor_0_9",
            "rango_mediano_variables","rango_maximo_variables","sugerencia_escalado",
            "pca_pc1","pca_pc1_pc2","pca_pc1_pc2_pc3"
        ])

    # Loop datasets
    for nombre, cfg in config_datasets.items():
        print(f"\n🔎 Generando reporte: {nombre.upper()}")
        try:
            X, y, _ = cargar_dataset(
                path=cfg["path"],
                clase_minoria=cfg.get("clase_minoria"),
                col_features=cfg.get("col_features"),
                col_target=cfg.get("col_target"),
                sep=cfg.get("sep", ","),
                header=cfg.get("header"),
                binarizar=cfg.get("binarizar", False),
                tipo=cfg.get("tipo", "tabular"),
                dataset_name=cfg.get("dataset_name")
            )

            # DataFrames seguros
            if isinstance(X, pd.DataFrame):
                X_df = X.copy()
            else:
                columnas_genericas = []
                indice_col = 0
                while indice_col < X.shape[1]:
                    columnas_genericas.append(f"col_{indice_col}")
                    indice_col += 1
                X_df = pd.DataFrame(X, columns=columnas_genericas)
            y_series = pd.Series(y)

            # Resumen de clases para tabla HTML
            conteo = pd.Series(y).value_counts()
            total = int(conteo.sum())
            resumen_clases = []
            for clase, n in conteo.items():
                porcentaje = float((n / max(1,total)) * 100.0)
                resumen_clases.append((clase, int(n), porcentaje))

            # Métricas
            metricas = calcular_metricas_basicas_dataframe(X_df, y_series)

            # PCA
            var_pca = calcular_varianza_explicada_pca(X_df, k=5)
            pc1 = float(var_pca[0]) if len(var_pca) > 0 else 0.0
            pc12 = float(sum(var_pca[:2])) if len(var_pca) >= 2 else pc1
            pc123 = float(sum(var_pca[:3])) if len(var_pca) >= 3 else pc12

            # Guardar en CSV consolidado
            with open(ruta_metricas_csv, "a", newline="", encoding="utf-8") as fcsv:
                escritor = csv.writer(fcsv)
                escritor.writerow([
                    nombre,
                    metricas["cantidad_muestras"],
                    metricas["cantidad_atributos"],
                    metricas["cantidad_clases"],
                    metricas["clase_minima_real"],
                    metricas["clase_mayoritaria_real"],
                    metricas["n_min"],
                    metricas["n_max"],
                    round(metricas["ratio_desequilibrio_max"], 6) if math.isfinite(metricas["ratio_desequilibrio_max"]) else "inf",
                    round(metricas["entropia_clases"], 6),
                    round(metricas["tamano_efectivo_clases"], 6),
                    round(metricas["porcentaje_faltantes"], 6),
                    round(metricas["porcentaje_duplicados"], 6),
                    round(metricas["asimetria_mediana"], 6),
                    round(metricas["curtosis_mediana"], 6),
                    round(metricas["porcentaje_variables_con_normalidad_no_rechazada"], 6),
                    round(metricas["correlacion_absoluta_maxima"], 6),
                    round(metricas["porcentaje_pares_correlacion_mayor_0_9"], 6),
                    round(metricas["rango_mediano_variables"], 6),
                    round(metricas["rango_maximo_variables"], 6),
                    metricas["sugerencia_escalado"],
                    round(pc1, 6),
                    round(pc12, 6),
                    round(pc123, 6)
                ])

            # Carpetas y figuras
            carpeta_dataset = carpeta_figuras / nombre.lower()
            carpeta_dataset.mkdir(exist_ok=True, parents=True)

            ruta_fig_clases = os.path.join(str(carpeta_dataset), f"{nombre.lower()}_clases.png")
            graficar_distribucion_clases(y, nombre_dataset=nombre, guardar_en=ruta_fig_clases)

            graficar_histograma_variables(X_df, nombre, str(carpeta_dataset), max_columnas=12)
            graficar_boxplot_variables(X_df, nombre, str(carpeta_dataset), max_columnas=12)
            graficar_mapa_correlaciones(X_df, nombre, str(carpeta_dataset))

            # Recolectar rutas de imágenes para el HTML
            rutas_figuras = []
            rutas_figuras.append(("Distribución de clases", ruta_fig_clases))

            # Agregar hist y box generados (primeras 12 numéricas)
            columnas_numericas = [c for c in X_df.columns if es_numerica(X_df[c])]
            limite = min(12, len(columnas_numericas))
            indice = 0
            while indice < limite:
                col = columnas_numericas[indice]
                rutas_figuras.append((f"Histograma: {col}", os.path.join(str(carpeta_dataset), f"{nombre}_hist_{col}.png")))
                rutas_figuras.append((f"Boxplot: {col}", os.path.join(str(carpeta_dataset), f"{nombre}_box_{col}.png")))
                indice += 1

            # Correlaciones (si hubo)
            if len(columnas_numericas) >= 2:
                rutas_figuras.append(("Matriz de correlaciones", os.path.join(str(carpeta_dataset), f"{nombre}_correlaciones.png")))

            # HTML por dataset
            ruta_html_dataset = carpeta_resultados / f"reporte_{nombre.lower()}.html"
            generar_html_dataset(
                nombre_dataset=nombre,
                resumen_clases=resumen_clases,
                metricas=metricas,
                varianza_pca=var_pca,
                rutas_figuras=rutas_figuras,
                ruta_html_salida=str(ruta_html_dataset)
            )

            # Agregar al índice
            rel = os.path.relpath(str(ruta_html_dataset), start=str(carpeta_resultados))
            filas_index.append(f"<li><a href='{rel}'>{nombre}</a></li>")
            print(f"   · OK -> {ruta_html_dataset}")

        except Exception as e:
            print(f"   · ERROR en {nombre}: {e}")

    filas_index.append("</ul>")
    filas_index.append("<hr/><p>Carpeta de figuras: <code>" + str(carpeta_figuras) + "</code></p>")
    filas_index.append("</body></html>")
    with open(ruta_index, "w", encoding="utf-8") as f:
        f.write("\n".join(filas_index))

    print("\n✅ Reporte unificado generado.")
    print("   - Índice:", ruta_index)
    print("   - Métricas consolidadas:", ruta_metricas_csv)
    print("   - Figuras por dataset en:", carpeta_figuras)

# -------------------------------------------------------------------
# EJECUCIÓN: llama con tus objetos reales
# -------------------------------------------------------------------
# Requiere que existan en tu notebook:
#  - config_datasets
#  - cargar_dataset(path, clase_minoria, col_features, col_target, sep, header, binarizar, tipo)
#  - graficar_distribucion_clases(y, nombre_dataset, guardar_en)
generar_reporte_unificado(config_datasets, cargar_dataset, graficar_distribucion_clases)



🔎 Generando reporte: SHUTTLE
   · ERROR en shuttle: "None of [Index(['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'], dtype='object')] are in the [columns]"

🔎 Generando reporte: WDBC
   · OK -> resultados_reporte_unificado_2025-11-10_0104\reporte_wdbc.html

🔎 Generando reporte: GLASS
   · OK -> resultados_reporte_unificado_2025-11-10_0104\reporte_glass.html

🔎 Generando reporte: HEART
   · OK -> resultados_reporte_unificado_2025-11-10_0104\reporte_heart.html

🔎 Generando reporte: IRIS
   · OK -> resultados_reporte_unificado_2025-11-10_0104\reporte_iris.html

🔎 Generando reporte: ECOLI
   · OK -> resultados_reporte_unificado_2025-11-10_0104\reporte_ecoli.html

✅ Reporte unificado generado.
   - Índice: resultados_reporte_unificado_2025-11-10_0104\index.html
   - Métricas consolidadas: resultados_reporte_unificado_2025-11-10_0104\metricas_eda_consolidado.csv
   - Figuras por dataset en: resultados_reporte_unificado_2025-11-10_0104\figuras


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.ensemble import IsolationForest

# Asumo que ya tenés:
# - config_datasets
# - cargar_dataset(path, clase_minoria, col_features, col_target, sep, header, binarizar, tipo, impute, na_values)
# Si tus funciones están en módulos, importalas antes.

def diagnosticar_outliers_shuttle(config_datasets, nombre_dataset="shuttle", ruta_salida="diagnosticos"):
    Path(ruta_salida).mkdir(exist_ok=True, parents=True)

    cfg = config_datasets[nombre_dataset]
    X, y, _ = cargar_dataset(
        path=cfg["path"],
        clase_minoria=cfg.get("clase_minoria"),
        col_features=cfg.get("col_features"),
        col_target=cfg.get("col_target"),
        sep=cfg.get("sep", ","),
        header=cfg.get("header"),
        binarizar=False,                 # Shuttle multiclase
        tipo=cfg.get("tipo", "tabular"),
        impute="median"
    )

    # X viene como ndarray (según tu cargar_dataset) -> armo DataFrame con nombres
    nombres_columnas = cfg.get("col_features")
    df = pd.DataFrame(X, columns=nombres_columnas)
    serie_clase = pd.Series(y, name="clase")

    # === 1) Resumen univariado: skew, kurtosis, % outliers por IQR ===
    lista_filas_resumen = []
    for nombre_columna in nombres_columnas:
        serie = df[nombre_columna].astype(float)

        q1 = float(np.percentile(serie, 25))
        q3 = float(np.percentile(serie, 75))
        iqr = q3 - q1
        limite_inferior = q1 - 1.5 * iqr
        limite_superior = q3 + 1.5 * iqr

        cantidad_outliers = int(((serie < limite_inferior) | (serie > limite_superior)).sum())
        porcentaje_outliers = 100.0 * cantidad_outliers / len(serie)

        asimetria = float(pd.Series(serie).skew())
        curtosis = float(pd.Series(serie).kurtosis())

        fila = {
            "variable": nombre_columna,
            "asimetria": asimetria,
            "curtosis": curtosis,
            "q1": q1,
            "q3": q3,
            "iqr": iqr,
            "limite_inferior": limite_inferior,
            "limite_superior": limite_superior,
            "cantidad_outliers_IQR": cantidad_outliers,
            "porcentaje_outliers_IQR": porcentaje_outliers
        }
        lista_filas_resumen.append(fila)

    resumen_univariado = pd.DataFrame(lista_filas_resumen).sort_values("porcentaje_outliers_IQR", ascending=False)
    resumen_univariado.to_csv(f"{ruta_salida}/shuttle_resumen_univariado.csv", index=False)

    # === 2) Gráficos univariados (histograma + boxplot) por variable ===
    for nombre_columna in nombres_columnas:
        serie = df[nombre_columna].astype(float)

        plt.figure(figsize=(8, 3))
        plt.hist(serie, bins=50)
        plt.title(f"Histograma - {nombre_columna}")
        plt.xlabel(nombre_columna)
        plt.ylabel("Frecuencia")
        plt.tight_layout()
        plt.savefig(f"{ruta_salida}/shuttle_hist_{nombre_columna}.png", dpi=200)
        plt.close()

        plt.figure(figsize=(6, 3))
        plt.boxplot(serie, vert=True, showfliers=True)
        plt.title(f"Boxplot - {nombre_columna}")
        plt.ylabel(nombre_columna)
        plt.tight_layout()
        plt.savefig(f"{ruta_salida}/shuttle_box_{nombre_columna}.png", dpi=200)
        plt.close()

    # === 3) Outliers por clase usando IQR (no borro, solo mido) ===
    lista_resumen_por_clase = []
    clases_unicas = np.unique(serie_clase.values)
    for clase_actual in clases_unicas:
        indice_clase = (serie_clase.values == clase_actual)
        subdf = df.loc[indice_clase]

        # porcentaje promedio de outliers IQR (promedio sobre columnas)
        porcentaje_por_variable = []
        for nombre_columna in nombres_columnas:
            serie = subdf[nombre_columna].astype(float)
            if len(serie) == 0:
                continue
            q1 = float(np.percentile(serie, 25))
            q3 = float(np.percentile(serie, 75))
            iqr = q3 - q1
            limite_inferior = q1 - 1.5 * iqr
            limite_superior = q3 + 1.5 * iqr
            cant = int(((serie < limite_inferior) | (serie > limite_superior)).sum())
            porcentaje = 100.0 * cant / len(serie)
            porcentaje_por_variable.append(porcentaje)
        promedio_porcentaje = float(np.mean(porcentaje_por_variable)) if len(porcentaje_por_variable) > 0 else 0.0

        lista_resumen_por_clase.append({
            "clase": clase_actual,
            "muestras_clase": int(indice_clase.sum()),
            "promedio_%_outliers_IQR": promedio_porcentaje
        })

    resumen_por_clase = pd.DataFrame(lista_resumen_por_clase).sort_values("muestras_clase", ascending=False)
    resumen_por_clase.to_csv(f"{ruta_salida}/shuttle_outliers_por_clase_IQR.csv", index=False)

    # === 4) IsolationForest por clase (solo para medir en la mayoritaria, no para borrar) ===
    # Nota: no uses este resultado para eliminar en clases minoritarias.
    if len(clases_unicas) > 0:
        # identifico clase mayoritaria
        conteo = pd.Series(serie_clase).value_counts()
        clase_mayoritaria = conteo.idxmax()
        indice_mayoritaria = (serie_clase.values == clase_mayoritaria)
        X_may = df.loc[indice_mayoritaria].values.astype(float)

        if X_may.shape[0] > 100:
            modelo_iso = IsolationForest(
                n_estimators=200,
                contamination=0.002,  # umbral MUY estricto: ~0.2%
                random_state=42
            )
            etiquetas = modelo_iso.fit_predict(X_may)
            porcentaje_anomalias = 100.0 * (etiquetas == -1).sum() / len(etiquetas)

            with open(f"{ruta_salida}/shuttle_isolationforest_mayoritaria.txt", "w", encoding="utf-8") as f:
                f.write(f"Clase mayoritaria: {clase_mayoritaria}\n")
                f.write(f"Muestras en clase mayoritaria: {len(etiquetas)}\n")
                f.write(f"Porcentaje marcado como anomalía (contamination=0.002): {porcentaje_anomalias:.4f}%\n")

    # === 5) Recomendación final en texto ===
    with open(f"{ruta_salida}/shuttle_recomendacion.txt", "w", encoding="utf-8") as f:
        f.write("Recomendación Shuttle:\n")
        f.write("- No eliminar outliers: los extremos reflejan transiciones reales del sistema.\n")
        f.write("- Si el modelo lo requiere, usar RobustScaler o winsorización leve (p0.1–p99.9) y validar.\n")
        f.write("- Evitar IsolationForest global. Si se limpia, hacerlo solo en clase mayoritaria y evaluar impacto.\n")

    return resumen_univariado, resumen_por_clase

# Ejecutar diagnóstico
resumen_univariado, resumen_por_clase = diagnosticar_outliers_shuttle(config_datasets, "shuttle", "diagnosticos")
resumen_univariado.head(), resumen_por_clase


KeyError: "None of [Index(['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'], dtype='object')] are in the [columns]"